## DATA ENGINEERING PIPELINE - INTRASTAT DECLARATION

Aim: Write a production ready data engineering pipeline using python and pandas.

Overview: Intrastat is a system that collects information relating to the trade of goods. This script will transform sample invoice data from a fictious company into a submissable Swedish intrastat declaration.

Task:

Below outlines the steps to be performed:

    01) Import the necessary libraries for the project.
    02) Define the functions that will faciliate the data engineering.
    03) Request intrastat commodity code list URL and read content into pd dataframe.
    04) Request ECB FX rates from URL, parse xml file and read data into pd dataframe.
    05) Cleanse and transform ex.rate data into pivot table.
    06) Read the sample intrastat data into a pd dataframe.
    07) Verify sample data using commodity code list.
    08) Apply daily exchange rate calculation on sample invoice values.
    09) Apply final transformations to intrastat output file. 
    10) Display the content of the prepared file.
    11) Export the content as an excel file, submissible to the Swedish stats authority.

#### Import Packages

In [5]:
import pandas as pd # Data analysis library.
import numpy as np # Array and matrice libary
import ssl # Secure sockets layer package.
import urllib # Url handling module.
import xml.etree.ElementTree as et # XML parsing library.
import datetime as dt # Datetime parsing library.

#### Define Functions

Functions to read, transform and export commodity code data.

In [6]:
def cc_read(url):
    # Function to request commodity code list and read url content into dataframe.
    try:
        # If URL is valid print confirmation.
        urllib.request.urlopen(url)
        print('Message: Requested URL is valid.')
        # Read data into pandas dataframe.
        df = pd.read_excel(url)
    except urllib.error.HTTPError as e:
        if e.code == 404:
            print('Error: Requested commodity code url is invalid.')
        else:
            print('Message: Requested commodity code url is valid.')
    return df

def cc_transform(df):
    # Function to convert commodity codes into cn8 format.
    df.iloc[:,0] = df.iloc[:,0].astype("str").str.pad(8, side='left', fillchar='0')
    # Rename first column to CN8. 
    df.columns.values[0] = 'CN8'
    return df

def cc_export(df):
    # Function to export CC codes to csv file 
    df.to_csv('CN8 Codes.csv', index=False)
    
def cc_rte_process(url):
    # Function to run full rte process and display csv output file.
    df_request = cc_read(url)
    df_transform = cc_transform(df_request)
    cc_export(df_transform)
    print('1. Commodity Code List')
    display(pd.read_csv('CN8 Codes.csv', dtype=str))
    return df_transform

Functions to read, transform and export foreign exchange rate data.

In [7]:
def fx_parse_xml(xml_obj, xml_child, xml_namespaces):
    # Function to parse xml content and read into dataframe. 
    try:
        xml_tree = et.parse(xml_obj)
        xml_root = xml_tree.getroot()
        # Find required tags and store data via list comprehension.
        rows = xml_root.findall('.//ex:Cube', namespaces=xml_namespaces)
        xml_data = [[row.get('time'), row.get('currency'), row.get('rate')] for row in rows]
        # Create columns for dataframe and read in content.
        df = pd.DataFrame(xml_data, columns = ['Date', 'Currency', 'Rate'])
        print('Message: Xml data parsing successful.')
    except et.ParseError:
            # Return empty dataframe if parse error.
            df = pd.DataFrame()
            print('Error: Xml data parsing failed.')
    return df    
    
def fx_create_pivot(df):
    # Function to create fx rate pivot table by date and currency.
    df_out = pd.pivot_table(df, index='Date', columns='Currency', values='Rate')
    # Add weekend dates missing from period to the table index.
    date_idx = pd.date_range(df['Date'].min(), df['Date'].max())
    df_out.index = pd.DatetimeIndex(df_out.index)
    df_out = df_out.reindex(date_idx)
    # Fill forward missing weekend fx rate values. 
    df_out = df_out.ffill(axis=0)
    df_out = df_out.sort_index(ascending=0)
    return df_out

def fx_read(url, xml_child, xml_namespaces):
    # Function to request fx rate url, parse content and read into dataframe.
    try:
        # Request url content. 
        xml_object = urllib.request.urlopen(url)
        df = fx_parse_xml(xml_object, xml_child, xml_namespaces)
    except urllib.error.HTTPError as e:
        if e.code == '404':
            # If URL is invalid create empty dataframe and print error.
            print('Error: Requested URL is invalid.')
        else:
            # If URL is valid read content into dataframe and print confirmation. 
            print('Message: Requested URL is valid.')
    return df

def fx_transform(df):
    # Function to fill empty fx dates, clean data and convert into pivot table. 
    df = df.ffill(axis=0)
    # Drop all other empty rows.
    df = df.dropna()
    # Create ex.rate pivot table.
    df['Rate'] = pd.to_numeric(df['Rate'])
    df_out = fx_create_pivot(df)
    return df_out

def fx_export(df):
    # Function to export to csv file 
    df.index.name = 'Date'
    df.to_csv('ECB FX Rates.csv', index=True)
    
def fx_rte_process(url, xml_child, xml_namespaces):
    # Function to run full rte process and display csv output file.
    df_read = fx_read(url, xml_child, xml_namespaces)
    df_transform = fx_transform(df_read)
    fx_export(df_transform)
    print('2. ECB FX Rate Table')
    display(pd.read_csv('ECB FX Rates.csv', dtype=str))
    return df_transform

Functions to read, check, transform and export source data.

In [8]:
def src_return_mot(mode):
    # Function to switch mot codes.
    mot_switch={'Sea':'1', 'Rail':'2','Road':'3', 'Air':'4'}
    return mot_switch.get(mode,"Invalid mode of transport\n")
    
def src_cc_check(df_src, df_cc):
    # Function to verify src commodity codes match official codes.
    df_out = pd.merge(df_src, df_cc, how='left', left_on='Commodity Code', right_on='CN8')
    df_out['CC Check'] = np.where(df_out['Commodity Code'] == df_out['CN8'], 'OK', '`ERROR')
    return df_out

def src_fx_convert(df_src, df_fx):
    # Function to perform fx rate conversion by shipping date.
    df_src['Shipping Date'] = df_src['Shipping Date'].astype("string")
    df_src['Shipping Date'] = pd.to_datetime(df_src['Shipping Date'], format="%d-%m-%Y")
    df_out = pd.merge(df_src, df_fx, how='left', left_on='Shipping Date', right_on=df_fx.index)
    df_out.rename(columns = {'SEK':'EUR to SEK'}, inplace = True)
    df_out['Net (SEK)'] = df_out['Net (EUR)'].astype('float').multiply(df_out['EUR to SEK'].astype('float'))    
    return df_out

def src_read(input_file_name):
    #Function to read intrastat source data.  
    df = pd.read_excel(input_file_name, dtype = str)
    try:
        # If file is read into data frame print confirmation.
        print('\n\nMessage: Source data successfully read.\n')
        print('3. Intrastat Source File')
        display(df)
    except pd.errors.ParserError:
        # If file is not read into data frame print error.
        print('Error: Source data could not be read.\n')
        sys.exit()
    return df

def src_analyse(df_src, df_cc, df_fx):
    # Function to analyse source data commodity codes and daily fx rate.  
    df_cc_check = src_cc_check(df_src, df_cc)
    df_fx_convert = src_fx_convert(df_cc_check, df_fx)
    return df_fx_convert

def src_transform(df_src):
    # Function to transform column data into format for intrastat submission.
    df_src['Mode of Transport'] = [src_return_mot(mode) for mode in df_src['Mode of Transport']] 
    df_src['Partner VAT'] = np.where(df_src['Transaction'] == 'B2C', 'QV999999999999', df_src['Partner VAT'])
    df_src['Mass (KG)'] = df_src['Mass (grams)'].astype('float').multiply(0.001)
    df_src["County of Origin"] = 'CN'
    df_src = df_src.drop(['Description_x', 'Mass (grams)', 'Shipping Date', 'Ship From', 'Incoterms', 'Transaction','CN8','SU', 'Description_y', 'CC Check', 'Net (EUR)', 'EUR to SEK' ], axis = 1)
    df_src = df_src[['Ship To', 'Commodity Code','Net (SEK)', 'Quantity', 'Mass (KG)', 'County of Origin', 'Mode of Transport', 'Partner VAT'  ]]    
    return df_src

def src_export(file_name, df):
    # Function to export to excel file.
    df.iloc[:,0] = df.iloc[:,0].astype("str")
    df.to_excel(file_name, index=False)
    print('\n\nMessage: Submission file successfully exported.\n')
    print('4. Intrastat Submission File')
    display(df)

def src_rate_process(input_file_name, df_cc, df_fx, output_file_name):
    # Function to perform rate process.
    df_read = src_read(input_file_name)
    df_analyse = src_analyse(df_read, df_cc, df_fx)
    df_transform = src_transform(df_analyse)
    src_export(output_file_name, df_transform)
    

#### Define Main

In [9]:
def main():
    # Define variables.
    input_file_name = 'Intrastat Dispatches Data Sample.xlsx'
    output_file_name = 'Intrastat Submission Sample.xlsx'
    cc_url = 'https://www.cbs.nl/-/media/cbsvooruwbedrijf/international-trade-in-goods/commoditycodes-2023.xlsx'
    fx_url = 'https://www.ecb.europa.eu/stats/eurofxref/eurofxref-hist-90d.xml'
    xml_namespaces = {'ex': 'http://www.ecb.int/vocabulary/2002-08-01/eurofxref'}
    xml_child = './/ex:Cube'
    
    # Disable security certificate checks for url requests.
    ssl._create_default_https_context = ssl._create_unverified_context
     
    #Read url references into data frames.
    df_cn8_codes = cc_rte_process(cc_url)
    df_fx_rates = fx_rte_process(fx_url, xml_child, xml_namespaces)
    
    #Run intrastat data check, fx conversion, transformation and export process.
    src_rate_process(input_file_name, df_cn8_codes, df_fx_rates, output_file_name)
    
# Define main as program entry point if script is running as standalone and not as module.
if __name__=="__main__":
    main()

Message: Requested URL is valid.
1. Commodity Code List


,CN8,SU,Description
0,01012100,p/st,Pure-bred breeding horses
1,01012910,p/st,Horses for slaughter
2,01012990,p/st,"Live horses (excl. for slaughter, pure-bred fo..."
3,01013000,p/st,Live asses
4,01019000,p/st,Live mules and hinnies
...,...,...,...
9750,97052900,-,Collections and collectors? pieces of zoologic...
9751,97053100,-,Collections and collectors? pieces of numismat...
9752,97053900,-,Collections and collectors? pieces of numismat...
9753,97061000,-,"Antiques, over 250 years old"


Message: Xml data parsing successful.
2. ECB FX Rate Table


,Date,AUD,BGN,BRL,CAD,CHF,CNY,CZK,DKK,GBP,...,NZD,PHP,PLN,RON,SEK,SGD,THB,TRY,USD,ZAR
0,2023-01-27,1.5289,1.9558,5.5104,1.4479,1.0017,7.369,23.826,7.4378,0.87885,...,1.6759,59.187,4.7085,4.8965,11.2108,1.4277,35.702,20.4365,1.0865,18.7185
1,2023-01-26,1.5308,1.9558,5.5572,1.4568,1.0002,7.3893,23.818,7.4383,0.87945,...,1.6799,59.301,4.7195,4.8818,11.1763,1.4292,35.687,20.4961,1.0895,18.665599999999998
2,2023-01-25,1.536,1.9558,5.569,1.4544,1.002,7.3778,23.808,7.4381,0.88248,...,1.6792,59.35,4.7158,4.9035,11.1335,1.4307,35.718,20.4658,1.0878,18.6436
3,2023-01-24,1.547,1.9558,5.6164,1.4517,1.0053,7.3642,23.874,7.4399,0.88368,...,1.6743,59.122,4.713,4.9171,11.0995,1.4337,35.609,20.4234,1.0858,18.7216
4,2023-01-23,1.5529,1.9558,5.6372,1.4523,1.0013,7.373,23.881,7.4393,0.8797,...,1.6778,59.345,4.7113,4.9202,11.1183,1.4328,35.63,20.4478,1.0871,18.714199999999998
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84,2022-11-04,1.5311,1.9558,4.9682,1.3351,0.9863,7.0894,24.422,7.4419,0.87478,...,1.6769,57.672,4.6825,4.8893,10.8538,1.3891,36.906,18.3845,0.9872,17.7783
85,2022-11-03,1.5517,1.9558,5.0262,1.3452,0.9889,7.1367,24.539,7.4433,0.87228,...,1.6957,57.463,4.709,4.9013,10.932,1.3878,37.091,18.1602,0.9753,17.9078
86,2022-11-02,1.5426,1.9558,5.0964,1.347,0.9861,7.2156,24.506,7.4431,0.861,...,1.6844,57.841,4.7035,4.912,10.9065,1.3983,37.314,18.4488,0.9908,17.98905
87,2022-11-01,1.5409,1.9558,5.1337,1.3469,0.9878,7.2165,24.484,7.4438,0.86058,...,1.6876,57.786,4.7053,4.9138,10.874,1.4017,37.45,18.5216,0.9947,17.9705




Message: Source data successfully read.

3. Intrastat Source File


,Commodity Code,Description,Mass (grams),Net (EUR),Quantity,Shipping Date,Ship From,Ship To,County of Origin,Mode of Transport,Incoterms,Transaction,Partner VAT
0,61012010,Men's or boys' overcoats,595,190,1,04-01-2023,SE,DE,China,Rail,DAP,B2C,Private Customer
1,61012090,Men's or boys' overcoats,678,150,1,01-01-2023,SE,NL,China,Sea,DDP,B2B,NL999999999999
2,61013010,Men's or boys' overcoats,704,175,1,05-01-2023,SE,ES,China,Road,DAP,B2C,Private Customer
3,61019080,Men's or boys' overcoats,844,135,1,07-01-2023,SE,FR,China,Air,DDP,B2B,FR999999999999
4,61021010,Women's or girls' overcoats,461,145,1,02-01-2023,SE,NL,China,Road,DDP,B2B,NL999999999999
5,61021090,Women's or girls' overcoats,589,160,1,03-01-2023,SE,BE,China,Air,DDP,B2B,BE999999999999
6,61022090,Women's or girls' overcoats,533,155,1,05-01-2023,SE,PT,China,Sea,DAP,B2C,Private Customer
7,61029010,Women's or girls' overcoats,406,180,1,06-01-2023,SE,IT,China,Rail,DAP,B2C,Private Customer




Message: Submission file successfully exported.

4. Intrastat Submission File


,Ship To,Commodity Code,Net (SEK),Quantity,Mass (KG),County of Origin,Mode of Transport,Partner VAT
0,DE,61012010,2121.4070,1,0.595,CN,2,QV999999999999
1,NL,61012090,1668.2700,1,0.678,CN,1,NL999999999999
2,ES,61013010,1957.3400,1,0.704,CN,3,QV999999999999
3,FR,61019080,1519.8300,1,0.844,CN,4,FR999999999999
4,NL,61021010,1619.1135,1,0.461,CN,3,NL999999999999
5,BE,61021090,1782.8800,1,0.589,CN,4,BE999999999999
6,PT,61022090,1733.6440,1,0.533,CN,1,QV999999999999
7,IT,61029010,2026.4400,1,0.406,CN,2,QV999999999999
